<a href="https://colab.research.google.com/github/bees4ever/ai-workshop-2022/blob/main/YOLO_Workshop_Capgemini_2022_Exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 5

Yolo Training.


The challenge of this task is to train and finetune an own Yolo Model. For this purpose a labeled dataset has been prepared.

## Step 1 - Download the data

Run the following command to download the data.

In [ ]:
!wget https://github.com/bees4ever/ai-workshop-2022/blob/main/yolo_sample/custom_yolo_masked_person_poc_all_v05.zip?raw=true -O custom_yolo_masked_person_poc_all_v05.zip
!unzip custom_yolo_masked_person_poc_all_v05.zip


## Step 2 Data Preprocessing

The labeles are available in POX XML format. We need to convert it to a YOLO understandable format. Some steps are prepared, but you also need to do invest some time to complete the transformation.

First, we do an extraction of data labels from .xml file to dataframe

In [ ]:
import shutil
import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile
import pandas as pd
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import numpy as np
import os

In [ ]:
annotations = sorted(glob('/content/custom_yolo_masked_person_poc_all_v05/labels/*.xml'))
# hint: parsedXML


df = []
cnt = 0
for file in annotations:
  prev_filename = file.split('/')[-1].split('.')[0] + '.jpeg'
  filename = str(cnt) + '.jpg'
  row = []
  parsedXML = ET.parse(file)
  img_width = int(float(parsedXML.getroot().find('size/width').text))
  img_height = int(float(parsedXML.getroot().find('size/height').text))

  for node in parsedXML.getroot().iter('object'):
    blood_cells = node.find('name').text
    xmin = int(float(node.find('bndbox/xmin').text))
    xmax = int(float(node.find('bndbox/xmax').text))
    ymin = int(float(node.find('bndbox/ymin').text))
    ymax = int(float(node.find('bndbox/ymax').text))

    row = [prev_filename, filename, blood_cells, xmin, xmax, ymin, ymax, img_width, img_height]
    df.append(row)
  cnt += 1

data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax', 'img_width', 'img_height'])

data[['prev_filename','filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax','img_width', 'img_height']].to_csv('/content/custom_yolo_masked_person.csv', index=False)


### Step 3: Process dataframe and generate the YOLO_V5 compatible labeles

*Hint, the data frame has following structure:* 

- filename : contains the name of the image
- cell_type: denotes the type of the cell
- xmin: x-coordinate of the bottom left part of the image
- xmax: x-coordinate of the top right part of the image
- ymin: y-coordinate of the bottom left part of the image
- ymax: y-coordinate of the top right part of the image
- labels : Encoded cell-type **(Yolo - label input-1)**
- width : width of that bbox
- height : height of that bbox
- x_center : bbox center (x-axis)
-	y_center : bbox center (y-axis)
-	x_center_norm	: x_center normalized (0-1) **(Yolo - label input-2)**
-	y_center_norm : y_center normalized (0-1) **(Yolo - label input-3)**
- width_norm : width normalized (0-1) **(Yolo - label input-4)**
-	height_norm : height normalized (0-1) **(Yolo - label input-5)**

In [ ]:
df = pd.read_csv('/content/custom_yolo_masked_person.csv')
df.head(30)

In [ ]:
#img_width = 640
#img_height = 480

def width(df):
  return int(df.xmax - df.xmin)
def height(df):
  return int(df.ymax - df.ymin)
def x_center(df):
  return int(df.xmin + (df.width/2))
def y_center(df):
  return int(df.ymin + (df.height/2))

df = pd.read_csv('/content/custom_yolo_masked_person.csv')

le = preprocessing.LabelEncoder()
le.fit(df['cell_type'])
print(le.classes_)
labels = le.transform(df['cell_type'])
df['labels'] = labels

df['width'] = df.apply(width, axis=1)
df['height'] = df.apply(height, axis=1)

df['x_center'] = df.apply(x_center, axis=1)
df['y_center'] = df.apply(y_center, axis=1)

df['x_center_norm'] = df['x_center']/df.img_width
df['width_norm'] = df['width']/df.img_width

df['y_center_norm'] = df['y_center'] / df.img_height
df['height_norm'] = df['height']/ df.img_height

df.head(30)

In [ ]:
#@title SAMPLE PLOT - shape (480, 640, 3)
fig = plt.figure()
import cv2
#add axes to the image
ax = fig.add_axes([0,0,1,1])

# read and plot the image
image = plt.imread('/content/custom_yolo_masked_person_poc_all_v05/images/ana-itonishvili-_meIjZYso8M-unsplash.jpeg')
plt.imshow(image)

# iterating over the image for different objects
for _,row in df[df.prev_filename == "ana-itonishvili-_meIjZYso8M-unsplash.jpeg"].iterrows():
    xmin = row.xmin
    xmax = row.xmax
    ymin = row.ymin
    ymax = row.ymax
    
    width = xmax - xmin
    height = ymax - ymin
    print(row)
    # assign different color to different classes of objects
    if row.cell_type == 'Person':
        edgecolor = 'r'
        ax.annotate('Person', xy=(xmax-40,ymin+20))
    elif row.cell_type == 'Masked':
        edgecolor = 'b'
        ax.annotate('Masked', xy=(xmax-40,ymin+20))

        
    # add bounding boxes to the image
    rect = patches.Rectangle((xmin,ymin), width, height, edgecolor = edgecolor, facecolor = 'none')
    
    ax.add_patch(rect)

# Splitting into training and validation datasets

In [ ]:
df_train, df_valid = model_selection.train_test_split(df, test_size=0.1, random_state=13, shuffle=True)
print(df_train.shape, df_valid.shape)

In [ ]:
os.mkdir('/content/custom_masked_person/')
os.mkdir('/content/custom_masked_person/images/')
os.mkdir('/content/custom_masked_person/images/train/')
os.mkdir('/content/custom_masked_person/images/valid/')

os.mkdir('/content/custom_masked_person/labels/')
os.mkdir('/content/custom_masked_person/labels/train/')
os.mkdir('/content/custom_masked_person/labels/valid/')

**STRUCTURE OF .txt FILE**

- One row per object
- Each row is class x_center y_center width height format.
- Box coordinates must be in normalized xywh format (from 0 - 1). If your boxes  are in pixels, divide x_center and width by image width, and y_center and height by image height.
- Class numbers are zero-indexed (start from 0).


In [ ]:
def segregate_data(df, img_path, train_img_path, train_label_path):
  filenames = []
  for filename in df.filename:
    filenames.append(filename)
  filenames = set(filenames)
  
  for filename in filenames:
    yolo_list = []

    for _,row in df[df.filename == filename].iterrows():
      yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    print(os.path.join(train_label_path,str(row.prev_filename.split('.')[0])+".txt"))
    txt_filename = os.path.join(train_label_path,str(row.prev_filename.split('.')[0])+".txt")
    # Save the .img & .txt files to the corresponding train and validation folders
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
    shutil.copyfile(os.path.join(img_path,row.prev_filename), os.path.join(train_img_path,row.prev_filename))

In [ ]:
%%time
src_img_path = "/content/custom_yolo_masked_person_poc_all_v05/images"

train_img_path = "/content/custom_masked_person/images/train/"
train_label_path = "/content/custom_masked_person/labels/train/"


valid_img_path = "/content/custom_masked_person/images/valid"
valid_label_path = "/content/custom_masked_person/labels/valid"

segregate_data(df_train, src_img_path, train_img_path, train_label_path)
segregate_data(df_valid, src_img_path, valid_img_path, valid_label_path)

In [ ]:
try:
  shutil.rmtree('/content/bcc/images/train/.ipynb_checkpoints')
except FileNotFoundError:
  pass

try:
  shutil.rmtree('/content/bcc/images/valid/.ipynb_checkpoints')
except FileNotFoundError:
  pass

try:
  shutil.rmtree('/content/bcc/labels/train/.ipynb_checkpoints')
except FileNotFoundError:
  pass

try:
  shutil.rmtree('/content/bcc/labels/valid/.ipynb_checkpoints')
except FileNotFoundError:
  pass

print("No. of Training images", len(os.listdir(train_img_path)))
print("No. of Training labels", len(os.listdir(train_label_path)))

print("No. of valid images", len(os.listdir(valid_img_path)))
print("No. of valid labels", len(os.listdir(valid_label_path)))

# Step 4: Training session

In [ ]:
!mkdir -p '/content/drive/My Drive/Machine Learning Projects/YOLO/'
!cp -r '/content/custom_masked_person' '/content/drive/My Drive/Machine Learning Projects/YOLO/'

Clone the yolo v5 repository.
More can be found at here : [yolo](https://github.com/ultralytics/yolov5)

In [ ]:
!git clone  'https://github.com/ultralytics/yolov5.git'

In [ ]:
!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies

# WE SHOULD CREATE A .yaml FILE AND THEN PLACE IT INSIDE THE yolov5 FOLDER

#**Contents of YAML file**

train: /content/custom_masked_person/images/train                    
val: /content/custom_masked_person/images/valid

nc: 2

names: ['Masked', 'Person']


In [ ]:
!rm 'custom_yolo_masked_person.yaml'
!echo -e 'train: /content/custom_masked_person/images/train\nval: /content/custom_masked_person/images/valid\n\nnc: 2\nnames: ['Masked', 'Person']' >> custom_yolo_masked_person.yaml
!cat 'custom_yolo_masked_person.yaml'

In [ ]:
shutil.copyfile('/content/custom_yolo_masked_person.yaml', '/content/yolov5/custom_yolo_masked_person.yaml')

#**Also edit the number of classes (nc) in the ./models/*.yaml file**

Choose the yolo model of your choice, here I chose yolov5s.yaml (yolo - small)


In [ ]:
!sed -i 's/nc: 80/nc: 2/g' ./yolov5/models/yolov5s.yaml

In [ ]:
!zip -r custom_masked_person.zip custom_masked_person

# Training command

**Training Parameters**

!python 
- <'location of train.py file'> 
- --img <'width of image'>
- --batch <'batch size'>
- --epochs <'no of epochs'>
- --data <'location of the .yaml file'>
- --cfg <'Which yolo configuration you want'>(yolov5s/yolov5m/yolov5l/yolov5x).yaml | (small, medium, large, xlarge)
- --name <'Name of the best model after training'>

**Metrics from Basic Training Process**

**No.of classes, No.of images, No.of targets, Precision (P), Recall (R), mean Average Precision (map)**

|Class | Images | Targets | P | R | mAP@.5 | mAP@.5:.95: |
|---|---|---|---|---|---|---|
| all   | 32    |     33 |   0.307 |       0.603  |      0.426 |      0.296|

**Metrics from our best Training Process**

**No.of classes, No.of images, No.of targets, Precision (P), Recall (R), mean Average Precision (map)**

| Class | Images | Targets | P | R | mAP@.5 | mAP@.5:.95: |
|---|---|---|---|---|---|---|
| all   | 32    |     33 |   0.60775 |       0.69485  |      0.65497 |     0.49628|


**Who is doing better??**



In [ ]:
%%time
#' basic training
!python yolov5/train.py --img 640 --batch 1 --epochs 200 --data yolov5/custom_yolo_masked_person.yaml --cfg yolov5/models/yolov5s.yaml --name CriminalDetection

Download example Hyper Parameter YAML file. 

In [ ]:
!wget https://raw.githubusercontent.com/bees4ever/ai-workshop-2022/main/hyp_evolve.yaml -O hyp_evolve.yaml

You should download it and edit it with some code editor and upload it again, to run an own trainings session.

In [ ]:
#' training with none default hyperparameter, this is the solution we can provide
#' TODO remove file and download the standart from yolo here, so students can play around with it

%%time
#' basic training
!python yolov5/train.py --img 640 --batch 8 --epochs 100 --data yolov5/custom_yolo_masked_person.yaml --cfg yolov5/models/yolov5s.yaml --name CriminalDetectionHyp  --hyp  /content/hyp_evolve.yaml

In [ ]:
# Start tensorboard (optional)
%load_ext tensorboard
%tensorboard --logdir yolov5/runs/

#**INFERENCE**

**Inference Parameters**

!python 
- <'location of detect.py file'> 
- --source <'location of image/ folder to predict'>
- --weight <'location of the saved best weights'>
- --output <'location of output files after prediction'>

In [ ]:
!python yolov5/detect.py --source /content/custom_masked_person/images/valid/ --weights '/content/yolov5/runs/train/CriminalDetection/weights/best.pt'  --save-conf

In [ ]:
!unzip prepared.zip

In [ ]:
!python yolov5/detect.py --source /content/prepared --weights '/content/yolov5/runs/train/CriminalDetection/weights/best.pt' 

In [ ]:
!python yolov5/detect.py --source /content/custom_yolo_masked_person_poc_all_v05/images/ --weights '/content/yolov5/runs/train/CriminalDetection/weights/best.pt' 

In [ ]:
disp_images = glob('/content/yolov5/runs/detect/exp3/*')
fig=plt.figure(figsize=(20*5, 28*5))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    img = np.random.choice(disp_images)
    img = plt.imread(img)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

# SINGLE IMAGE PREDICTIONS


In [ ]:
output = !python yolov5/detect.py --source /content/custom_yolo_masked_person_poc_all_v05/images/dave-spiess-QevEgXYHpzs-unsplash.jpeg --weights '/content/yolov5/runs/train/CriminalDetection/weights/best.pt'
print(output)

In [ ]:
#' export all predictions

!zip -r detections.zip yolov5/runs/detect/


# You need these files, if you wish to move the model to production

## Files

In [ ]:
shutil.copyfile('/content/yolov5/detect.py', '/content/drive/My Drive/Machine Learning Projects/YOLO/SOURCE/detect.py')
shutil.copyfile('/content/yolov5/requirements.txt', '/content/drive/My Drive/Machine Learning Projects/YOLO/SOURCE/requirements.txt')
shutil.copyfile('/content/yolov5/runs/train/CriminalDetection/weights/best.pt', '/content/drive/My Drive/Machine Learning Projects/YOLO/SOURCE/best.pt')



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Folder

In [ ]:
!cp -r '/content/yolov5/models' '/content/drive/My Drive/Machine Learning Projects/YOLO/SOURCE/'
!cp -r '/content/yolov5/utils' '/content/drive/My Drive/Machine Learning Projects/YOLO/SOURCE/'
